## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using LSTM or Bi-LSTM + SGD + One pass 

## Modifications:
- Reward Penalty based Weighted Ensemble [RPWE]


## Date: 23 Sept 2021 at 10:10 AM

In [9]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [11]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [12]:
#======================================================
# RESP BELT Feature Extraction (Wavelet Features)
#======================================================

def extract_resp_belt_features(raw_data):
    features =[]
    resp_belt = raw_data
    list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features

In [13]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [14]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [15]:
# #============================
# # Bi-directional LSTM model 
# #============================
# def create_model(x):
#     model = Sequential() 
    
#     model.add(Bidirectional(LSTM(units = 30, return_sequences = True, input_shape = (1,x.shape[1]))))

#     model.add(Bidirectional(LSTM(units = 30, activation='sigmoid',return_sequences = True)))
    
#     model.add(Dense(1, activation='sigmoid'))
    
#     opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    
#     model.compile(loss='binary_crossentropy',
#                   optimizer=opt,
#                   metrics=['accuracy'])
    
    
# #     print(model.summary())
    
#     return model

In [16]:
#==================================
# LSTM model
#==================================
def create_model(x):
    model = Sequential() 
    
    model.add(LSTM(units = 2, activation='sigmoid' ,return_sequences = True, input_shape = (1,x.shape[1])))

    model.add(LSTM(units = 2, activation='sigmoid',return_sequences = True))
    
    model.add(Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    
    print(model.summary())
    
    return model

In [18]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((8064/60)*segment_in_sec)  #8064 is for 60 sec record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers


optimizer= 'SGD' #optimizer
classifier = 'LSTM_RPWE'+str(optimizer)
indx = 0
c=0


participant = 32
videos = 40

global eeg_emotion, eda_emotion, resp_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
resp_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()


eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()


resp_acc_val = metrics.Accuracy() #Accuracy
resp_f1m_val = metrics.F1() #F1 measure  
resp_acc_aro = metrics.Accuracy() #Accuracy
resp_f1m_aro = metrics.F1() #F1 measure
resp_roc_val = metrics.ROCAUC()
resp_roc_aro = metrics.ROCAUC()
resp_mcc_val = metrics.MCC()
resp_mcc_aro = metrics.MCC()

val_MSE = []
aro_MSE = []



 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
resp_cm_val  = metrics.ConfusionMatrix()
resp_cm_aro  = metrics.ConfusionMatrix()


    
itr = 0 #controls the learning rate


#=======================================
# MAIN Loop STARTS HERE
#=======================================
for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/DEAP_data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # Resp data read from files
        ##===================================================
        def resp_data(p):
            file_resp = '/home/gp/Desktop/MER_arin/DEAP_data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
            print(file_resp)
            resp_sig = pd.read_csv(file_resp,sep=',', header = None, engine='python')
            return resp_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_resp = resp_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            resp_sig = grand_resp.loc[grand_resp.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            val = eeg_sig.iloc[0,8067]
            aro = eeg_sig.iloc[0,8068]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            resp_val_prdt=[]
            resp_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_resp = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,32),range(3,8067)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(3,8067)]) #keeping only eda signals
            resp_sig = np.array(resp_sig.iloc[:,range(3,8067)]) #keeping only resp signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_resp.fit(resp_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_resp = slider_resp.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                features_eeg = eeg_features(window_data_eeg)
                eeg = np.array([features_eeg])  #EEG raw feature vector
                x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                eda_features = extract_eda_features(np.array(window_data_eda))
                eda = np.array([eda_features]) #EDA raw feature vector
                x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from Resp belt
                #=================================================

                resp_features = extract_resp_belt_features(np.array(window_data_resp))
                resp = np.array([resp_features]) #RESP BELT raw feature vector
                x_resp = preprocessing.normalize(resp) #RESP BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('RESP BELT Feature shape{}:'.format(x_resp.shape))

                    #========================
                    # For EEG data model
                    #========================
                    eeg_model_val = create_model(x_eeg)

                    eeg_model_aro = create_model(x_eeg)

                    #========================
                    # For EDA data model
                    #========================
                    eda_model_val = create_model(x_eda)

                    eda_model_aro = create_model(x_eda)

                    #==============================
                    # For Resp Belt data Model
                    #==============================
                    resp_model_val = create_model(x_resp)

                    resp_model_aro = create_model(x_resp)

                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                if c == 0: #For the first time model will return 0 or None
                    tmp_eeg_val = np.array([0])
                    tmp_eeg_aro = np.array([0])
                    
                    tmp_eda_val = np.array([0])
                    tmp_eda_aro = np.array([0])
                    
                    tmp_resp_val = np.array([0])
                    tmp_resp_aro = np.array([0])
                    
                    
                    y_pred_val_eeg = np.array([2])
                    y_pred_aro_eeg = np.array([2])
                    
                    y_pred_val_eda = np.array([2])
                    y_pred_aro_eda = np.array([2])
                    
                    y_pred_val_resp = np.array([2])
                    y_pred_aro_resp = np.array([2])
                    
                    eeg_model_val.fit(np.array([x_eeg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    resp_model_val.fit(np.array([x_resp]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    resp_model_aro.fit(np.array([x_resp]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    c=c+1
                    
                else: 
                    
                    tmp_eeg_val  = eeg_model_val.predict(np.array([x_eeg]))
                    tmp_eeg_aro = eeg_model_aro.predict(np.array([x_eeg]))
                    
                    tmp_eda_val = eda_model_val.predict(np.array([x_eda]))
                    tmp_eda_aro  = eda_model_aro.predict(np.array([x_eda]))
                    
                    tmp_resp_val = resp_model_val.predict(np.array([x_resp]))
                    tmp_resp_aro = resp_model_aro.predict(np.array([x_resp]))
                    
                    #Binary output scaling 1 if x > 0.5 else 0
                    y_pred_val_eeg = [1 if tmp_eeg_val > 0.5 else 0]
                    y_pred_aro_eeg = [1 if tmp_eeg_aro > 0.5 else 0]
                    
                    y_pred_val_eda = [1 if tmp_eda_val > 0.5 else 0]
                    y_pred_aro_eda = [1 if tmp_eda_aro > 0.5 else 0]
                    
                    y_pred_val_resp = [1 if tmp_resp_val > 0.5 else 0]
                    y_pred_aro_resp = [1 if tmp_resp_aro > 0.5 else 0]
                    
                    #Fit the model once
                    eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    resp_model_val.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    resp_model_aro.fit(np.array([x_resp]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    

                #===========================================
                # Performance matric update
                #===========================================
                
                #Valence classification EEG

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg[0])  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg[0]) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                #Arousal classification EEG 

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg[0]) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


                #Valence classification EDA

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda[0])  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda[0]) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])
                

                #Arousal classification EDA
                
                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda[0])  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda[0]) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

                #Valence classification Resp Belt

                resp_acc_val = resp_acc_val.update(y_act_val[0], y_pred_val_resp[0])  # update the accuracy metric

                resp_f1m_val = resp_f1m_val.update(y_act_val[0], y_pred_val_resp[0]) #update f1 measure metric
                
                resp_roc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                
                resp_mcc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                

                #Arousal classification Resp Belt

                resp_acc_aro = resp_acc_aro.update(y_act_aro[0], y_pred_aro_resp[0])  # update the accuracy metric

                resp_f1m_aro = resp_f1m_aro.update(y_act_aro[0], y_pred_aro_resp[0]) #update f1 measure metric
                
                resp_roc_aro = resp_roc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                
                resp_mcc_aro = resp_mcc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))[0][0][0]
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))[0][0][0]
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))[0][0][0]
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))[0][0][0]
            
            resp_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_resp_val))[0][0][0]
            resp_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_resp_aro))[0][0][0]
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, resp_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, resp_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            resp_cm_val = resp_cm_val.update(y_act_val[0], y_pred_val_resp[0])
            resp_cm_varo = resp_cm_aro.update(y_act_aro[0], y_pred_aro_resp[0])            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg[0],y_pred_aro_eeg[0]]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda[0], y_pred_aro_eda[0]]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_resp[0], y_pred_aro_resp[0]]) #appending valence & arousal predicted resp

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg[0], y_act_aro[0], y_pred_aro_eeg[0]]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda[0], y_act_aro[0], y_pred_aro_eda[0]]))
            
            resp_emotion.append(np.array([p,v, resp_acc_val.get(), resp_f1m_val.get(), resp_roc_val.get(), resp_mcc_val.get(),
                                          resp_acc_aro.get(),resp_f1m_aro.get(), resp_roc_aro.get(), resp_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_resp[0], y_act_aro[0], y_pred_aro_resp[0]]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
            print('Val MSE',val_MSE)
            print('Aro MSE',aro_MSE)
            # print('------------------------------')
            print('Beta_Val:', beta_val)
            print('Beta_Aro:', beta_aro)
            print('------------------------------')
            print('W Val:', w_val)
            print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('RESP BELT')

            # print("Valence Accuracy:{}".format(round(resp_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(resp_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(resp_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(resp_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(resp_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(resp_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(resp_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(resp_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/1_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:1
EEG Feature shape(1, 1152):
EDA Feature shape(1, 48):
RESP BELT Feature shape(1, 48):
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 1, 2)              9240      
_________________________________________________________________
lstm_13 (LSTM)               (None, 1, 2)              40        
_________________________________________________________________
dense_6 (Dense)              (None, 1, 1)              3         
Total params: 9,283
Trainable params: 9,283
Non-trainable params: 0
______________________________

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.06870334595441818, 0.06318175047636032, 0.05366017669439316]
Aro MSE [0.09265351295471191, 0.07026930153369904, 0.08727750182151794]
Beta_Val: [0.00629156 0.00570256 0.00458372]
Beta_Aro: [0.00900212 0.00649959 0.00845004]
------------------------------
W Val: [0.33390212 0.33346686 0.33263102]
W Aro: [0.33407803 0.33224917 0.3336728 ]
MER model
Valence Accuracy:1.0
Valence F1 score:1.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:0.6667
Arousal F1 score:0.8
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:4
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.17474307119846344, 0.1833719313144683

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.10685773938894272, 0.11010892689228058, 0.11806032806634903]
Aro MSE [0.04095184803009033, 0.04819035530090332, 0.04233542084693909]
Beta_Val: [-0.02763959 -0.03027098 -0.03572874]
Beta_Aro: [0.00321279 0.00380754 0.00333016]
------------------------------
W Val: [0.34507434 0.33590023 0.31902543]
W Aro: [0.35017579 0.30860167 0.34122254]
MER model
Valence Accuracy:0.6154
Valence F1 score:0.7368
Valence ROC score:0.5833
Valence MCC score:0.3118
Arousal Accuracy:0.7692
Arousal F1 score:0.6667
Arousal ROC score:0.7639
Arousal MCC score:0.5007
===============================xxxx================================
Person: 1 Video:14
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.08966378122568

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.0656028687953949, 0.0646098181605339, 0.06326578557491302]
Aro MSE [0.1285354197025299, 0.13351373374462128, 0.12964142858982086]
Beta_Val: [-0.00873805 -0.0095243  -0.0109616 ]
Beta_Aro: [-0.04185567 -0.041795   -0.0419129 ]
------------------------------
W Val: [0.34873774 0.33645853 0.31480373]
W Aro: [0.34376119 0.31793548 0.33830333]
MER model
Valence Accuracy:0.6522
Valence F1 score:0.7333
Valence ROC score:0.6151
Valence MCC score:0.2441
Arousal Accuracy:0.6087
Arousal F1 score:0.5714
Arousal ROC score:0.6061
Arousal MCC score:0.2137
===============================xxxx================================
Person: 1 Video:24
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.05577803775668

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.10173927992582321, 0.10229331254959106, 0.10468960553407669]
Aro MSE [0.144842267036438, 0.1469976156949997, 0.14533056318759918]
Beta_Val: [-0.02970761 -0.03016458 -0.03136089]
Beta_Aro: [-0.04917818 -0.04791836 -0.04897394]
------------------------------
W Val: [0.35145415 0.33695093 0.31159492]
W Aro: [0.34026384 0.3231025  0.33663366]
MER model
Valence Accuracy:0.6364
Valence F1 score:0.7143
Valence ROC score:0.6167
Valence MCC score:0.2609
Arousal Accuracy:0.6061
Arousal F1 score:0.6486
Arousal ROC score:0.6029
Arousal MCC score:0.2106
===============================xxxx================================
Person: 1 Video:34
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.08546410501003

...

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.04530820623040199, 0.045401960611343384, 0.04687932878732681]
Aro MSE [0.06079835444688797, 0.06188802421092987, 0.06073055416345596]
Beta_Val: [-0.01091714 -0.01098606 -0.01154137]
Beta_Aro: [-0.01482142 -0.01506939 -0.01476203]
------------------------------
W Val: [0.35008805 0.33204883 0.31786312]
W Aro: [0.33746561 0.33228564 0.33024875]
MER model
Valence Accuracy:0.671
Valence F1 score:0.7224
Valence ROC score:0.6585
Valence MCC score:0.3265
Arousal Accuracy:0.6863
Arousal F1 score:0.7449
Arousal ROC score:0.667
Arousal MCC score:0.3481
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/32_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_d

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.10048232972621918, 0.10061012208461761, 0.10113949328660965]
Aro MSE [0.1498754620552063, 0.14934583008289337, 0.149669349193573]
Beta_Val: [-0.02157613 -0.0216128  -0.021355  ]
Beta_Aro: [-0.04136871 -0.04082428 -0.04130202]
------------------------------
W Val: [0.34993649 0.33180809 0.31825542]
W Aro: [0.33737395 0.33244896 0.3301771 ]
MER model
Valence Accuracy:0.6701
Valence F1 score:0.7216
Valence ROC score:0.6576
Valence MCC score:0.3241
Arousal Accuracy:0.6845
Arousal F1 score:0.7438
Arousal ROC score:0.665
Arousal MCC score:0.3447
===============================xxxx================================
Person: 32 Video:10
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.13467425107955

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.044822365045547485, 0.0448470339179039, 0.04536435753107071]
Aro MSE [0.06237561255693436, 0.06273553520441055, 0.0623655691742897]
Beta_Val: [0.00336027 0.00335268 0.0033932 ]
Beta_Aro: [-0.00036859 -0.00018957 -0.00039445]
------------------------------
W Val: [0.34962805 0.33152781 0.31884414]
W Aro: [0.33714224 0.3330445  0.32981326]
MER model
Valence Accuracy:0.6712
Valence F1 score:0.7236
Valence ROC score:0.658
Valence MCC score:0.3252
Arousal Accuracy:0.6847
Arousal F1 score:0.7447
Arousal ROC score:0.6645
Arousal MCC score:0.3443
===============================xxxx================================
Person: 32 Video:20
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.038763053715229

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.15522141754627228, 0.1550160050392151, 0.15535159409046173]
Aro MSE [0.054236024618148804, 0.05439446121454239, 0.05422275513410568]
Beta_Val: [-0.0526116  -0.05252129 -0.05250297]
Beta_Aro: [-0.00749414 -0.00748771 -0.00755601]
------------------------------
W Val: [0.34941543 0.33137649 0.31920808]
W Aro: [0.33701979 0.33340096 0.32957925]
MER model
Valence Accuracy:0.6698
Valence F1 score:0.7227
Valence ROC score:0.6566
Valence MCC score:0.3229
Arousal Accuracy:0.6848
Arousal F1 score:0.7455
Arousal ROC score:0.664
Arousal MCC score:0.3437
===============================xxxx================================
Person: 32 Video:30
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.13033729791

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.03159739077091217, 0.031726449728012085, 0.03279738873243332]
Aro MSE [0.0222723800688982, 0.022270794957876205, 0.0221781674772501]
Beta_Val: [0.00205693 0.00206926 0.00206979]
Beta_Aro: [0.00127576 0.00129141 0.0012758 ]
------------------------------
W Val: [0.34972521 0.33159654 0.31867826]
W Aro: [0.33700196 0.33348936 0.32950868]
MER model
Valence Accuracy:0.6716
Valence F1 score:0.7222
Valence ROC score:0.6597
Valence MCC score:0.3289
Arousal Accuracy:0.6865
Arousal F1 score:0.748
Arousal ROC score:0.6647
Arousal MCC score:0.3456
===============================xxxx================================
Person: 32 Video:40
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.2516585886478424,

In [19]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    313    259
      1    162    546
EEG aro CM:
             0      1
      0    283    260
      1    141    596
EDA Val CM:
             0      1
      0    314    258
      1    162    546
EDA aro CM:
             0      1
      0    282    261
      1    141    596
RESP Val CM:
             0      1
      0    312    260
      1    159    549
RESP Aro CM:
             0      1
      0    283    260
      1    142    595
ReMECS Val CM:
             0      1
      0    313    259
      1    162    546
ReMECS aro CM:
             0      1
      0    283    260
      1    142    595


In [20]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# RESP BELT
#=============================
t_resp_emotion =pd.DataFrame(resp_emotion) 
t_resp_emotion = t_resp_emotion.iloc[:,range(2,10)]
t_resp_emotion_avg = np.array(np.mean(t_resp_emotion))
t_resp_emotion_std = np.array(np.std(t_resp_emotion))

print('Resp Belt result Average:', t_resp_emotion_avg)
print('Resp Belt result std:', t_resp_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.63084926 0.68282084 0.61685514 0.24122102 0.63989577 0.67778124
 0.6255612  0.26125854]
EEG result std: [0.02027649 0.02586518 0.03269852 0.03127101 0.02860377 0.05942716
 0.04066869 0.04576143]
EDA result Average: [0.62998638 0.68218141 0.61595483 0.23939905 0.6446062  0.68480124
 0.62853771 0.26858814]
EDA result std: [0.02048326 0.02585913 0.03274363 0.03143381 0.02317282 0.05321601
 0.04045798 0.04104318]
Resp Belt result Average: [0.62768061 0.6816379  0.61310394 0.61310394 0.64157805 0.6801883
 0.62694077 0.26427033]
Resp Belt result std: [0.02121239 0.02599549 0.03288619 0.03288619 0.02661263 0.05753042
 0.04038151 0.04383851]
MER result Average: [0.66070904 0.70928579 0.64550806 0.30120936 0.67908821 0.71419573
 0.66377977 0.34133183]
MER result std: [0.02263941 0.0262052  0.03573413 0.0370135  0.0258623  0.05009947
 0.04325683 0.04571834]


In [22]:
time_stamp = datetime.datetime.now()
run =1
algo= 'LSTM/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/12DEAP_RPWE/Results_RPWE_DEAP/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_RESP_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)
